In [ ]:
#%load_ext wurlitzer
#^^^ the wurlitzer extension is used to capture C/C++ output to be displayed in the notebook
#^^^ this is very useful for debugging, but it doesn't work on windows

#Python 2/3 compatibility
from __future__ import print_function,division

import time
from klampt import *
from klampt.math import vectorops,so3,se3
from klampt import vis
from klampt.model.create import primitives
from IPython.display import clear_output
from klampt.vis.ipython import EditConfig,EditPoint,EditTransform

world = WorldModel()
world.loadFile("../data/athlete_plane.xml")
vis.add("world",world)

vis.add("sphere",[1,0,2],color=(0,1,0),size=0.5)
#vis.add("sphere",primitives.sphere(0.5,[1,0,2]))
#vis.setColor("sphere",0,1,0)
vis.add("ghost",world.robot(0).getConfig(),color=(0,1,0,0.5))

vis.show()

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

In [ ]:
q = world.robot(0).getConfig()
#getConfig() returns a copy of the robot's current configuration, so these changes
#aren't reflected in the WorldModel...
q[6] = 2
q[2] = 0.5
#until this call here
world.robot(0).setConfig(q)

#WAIT? why didn't the visualization change?
#... Because the WorldModel knows nothing about the visualization...

In [ ]:
#... so we need to do this manually.
#Use a kvis.update() call to push the current state of the world to the visualization
vis.update()

In [ ]:
#do a little animation... note the vis.update calls
q = world.robot(0).getConfig()
for i in range(20):
    q[2] = i*0.1
    world.robot(0).setConfig(q)
    vis.update()
    time.sleep(0.1)

In [ ]:
#can move the camera using setViewport() and setViewport()
cam = vis.getViewport()
print(cam)
cam['position']['x'] += 1.5
vis.setViewport(cam)

In [ ]:
#add some "extras"

vis.addText("some_text",text="hello",position=(10,10))
vis.add("sphere1",[1,0,0.5],size=0.3)
vis.setColor("sphere1",1,0,0,0.25)
q = [0.0]*world.robot(0).numLinks()
vis.add('ghost',q)
vis.setColor('ghost',0,1,0,0.5)

In [ ]:
#items can be updated by calling add again, setItemConfig, or setAttribute
vis.addText("some_text",text="hello modified")
vis.setAttribute("sphere1",'size',0.5)
vis.add("xform",(so3.identity(),[1,0,0]),length=0.5,width=3)

In [ ]:
#you can create multiple KlamptWidget objects
window2 = vis.createWindow()

vis.add("world",world)
vis.show()
print([a['name'] for a in vis.nativeWindow().scene['object']['children']])

In [ ]:
vis.lock()
vis.add('ghost',[0.0]*len(world.robot(0).getConfig()))
vis.setColor('ghost',0,1,0,0.5)
vis.add("xform",(so3.identity(),[1,0,0.5]))
vis.add("point",[0.5,0,1],color=(1,0,0))
vis.unlock()

print([a['name'] for a in vis.nativeWindow().scene['object']['children']])

In [ ]:
#and you can add editors
vis.edit('ghost')
vis.edit('point')
vis.edit('xform')


In [ ]:
#you can even load multiple worlds
world2 = WorldModel()
world2.loadFile('../data/hubo_plane.xml')
window4 = vis.createWindow()
vis.add("world",world2)

#here's how you show a playback widget 
from klampt.vis.ipython import Playback
playback_widget3 = Playback(vis.nativeWindow())
display(playback_widget3)
vis.show()

In [ ]:
#you can play around with these hooks to the playback widget
framerate = 30
movespeed = 1.0
robot = world2.robot(0)
q0 = robot.getConfig()
def advance():
    q = robot.getConfig()
    q[9] += movespeed/framerate
    robot.setConfig(q)
def reset():
    robot.setConfig(q0)
playback_widget3.advance = advance
playback_widget3.reset = reset
playback_widget3.framerate = framerate
playback_widget3.maxframes = 30

In [ ]:
#If the Jupyter notebook is running on your own computer, and you have OpenGL, you can
#still use the standard Klamp't vis functions
vis.kill()
vis.init(["PyQt","GLUT"])
vis.add("world",world)
vis.show()